# Машинное обучение. Практическая работа

В этой практической работе четыре обязательные задачи.

Они помогут понять, что вы действительно усвоили материал модуля. 


Удачи!

## Цели практической работы

Научиться:
- обучать модели, основанные на деревьях решений;
- научиться оценивать и увеличивать их качество.

## Что входит в практическую работу

1. Загрузите датасет и ознакомьтесь с ним.
2. Подготовьте базовую модель дерева решений и измерьте её качество.
3. Подготовьте базовую модель случайного леса и измерьте её качество.
4. Увеличьте точность модели случайного леса на тестовых данных.
5. Проведите анализ влияния признаков на модель.

## Что оценивается

* Выполнены все четыре задачи. Для каждой:
 * в коде нет ручных перечислений, все действия автоматизированы;
 * результаты вычислений и применённых операций корректны;
 * ответы на вопросы, где требуется, корректны и обоснованы; 
 * код читабелен: переменным даны осмысленные названия, соблюдены отступы и правила расстановки пробелов; стилизация кода соответствует рекомендациям [PEP 8](https://pythonworld.ru/osnovy/pep-8-rukovodstvo-po-napisaniyu-koda-na-python.html).

* Репозиторий проекта оформлен корректно:
 * содержит осмысленные коммиты, содержащие конкретные реализованные фичи;
 * ветки названы согласно назначению;
 * файлы, не связанные с проектом, не хранятся в репозитории.


## Как отправить работу на проверку

Сдайте практическую работу этого модуля через систему контроля версий Git сервиса Skillbox GitLab. После загрузки работы на проверку напишите об этом в личном кабинете своему куратору.

## Обязательные задачи

### Описание датасета:
- `id`— идентификатор записи;
- `is_manufacturer_name`— признак производителя автомобиля;

- `region_*`— регион;
- `x0_*`— тип топлива;
- `manufacturer_*`— производитель;
- `short_model_*`— сокращённая модель автомобиля;
- `title_status_*`— статус;
- `transmission_*`— коробка передач;
- `state_*`— штат;
- `age_category_*`— возрастная категория автомобиля;

- `std_scaled_odometer`— количество пройденных миль (после стандартизации);
- `year_std`— год выпуска (после стандартизации);
- `lat_std`— широта (после стандартизации);
- `long_std`— долгота (после стандартизации);
- `odometer/price_std`— отношение стоимости к пробегу автомобиля (после стандартизации);
- `desc_len_std`— количество символов в тексте объявления о продаже (после стандартизации);
- `model_in_desc_std`— количество наименований модели автомобиля в тексте объявления о продаже (после стандартизации);
- `model_len_std`— длина наименования автомобиля (после стандартизации);
- `model_word_count_std`— количество слов в наименовании автомобиля (после стандартизации);
- `month_std`— номер месяца размещения объявления о продаже автомобиля (после стандартизации);
- `dayofweek_std`— день недели размещения объявления о продаже автомобиля (после стандартизации);
- `diff_years_std`— количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля (после стандартизации);

- `price`— стоимость;
- `price_category`— категория цены.

1. Подготовка базовой модели

Обучите простую модель классификации с помощью DecisionTreeClassifier на данных из датасета vehicles_dataset_prepared.csv. Для этого сделайте шаги:

1. Обучите модель дерева решений с зафиксированным random_state на тренировочной выборке.
2. Сделайте предикт на тестовой выборке.
3. Замерьте точность на тестовой выборке и выведите матрицу ошибок. 
4. Удалите фичи с нулевыми весами по feature_importance из тренировочной и тестовой выборок.
5. Заново обучите модель и измерьте качество.

In [182]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix 

In [184]:
df = pd.read_csv('vehicles_dataset_prepared.csv')

df_prepared = df.copy()
df_prepared = df_prepared.drop(['price', 'odometer/price_std'], axis=1)

x = df_prepared.drop(['price_category'], axis=1)
y = df_prepared['price_category']

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42
)

In [185]:
# Ваш код здесь 
clf = DecisionTreeClassifier(random_state=42)
clf.fit(x_train, y_train)

predicted_train = clf.predict(x_train)
predicted_test = clf.predict(x_test)

print(accuracy_score(y_train, predicted_train))
print(accuracy_score(y_test, predicted_test))
print(confusion_matrix(y_test, predicted_test))

1.0
0.6704781704781705
[[738  54 205]
 [ 46 688 219]
 [198 229 509]]


In [186]:
f_imp_list = list(zip(x_train. columns, clf.feature_importances_))
f_imp_list.sort(key=lambda x: x[1], reverse=True)

importances = clf.feature_importances_
non_zero_mask = importances > 0

x_train_reduced = x_train.loc[:, non_zero_mask]
x_test_reduced = x_test.loc[:, non_zero_mask]

In [187]:
clf_reduced = DecisionTreeClassifier(random_state=42)

clf_reduced.fit(x_train_reduced, y_train)

predicted_train = clf_reduced.predict(x_train_reduced)
predicted_test = clf_reduced.predict(x_test_reduced)

print(accuracy_score(y_train, predicted_train))
print(accuracy_score(y_test, predicted_test))

1.0
0.6725571725571725


2. Подготовка модели случайного леса

Обучите простую модель классификации с помощью RandomForestClassifier. Для этого на новых урезанных семплах тренировочной и тестовой выборок обучите модель случайного леса с зафиксированным random_state=50. 

In [189]:
# Ваш код здесь
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(random_state=42)

clf_rf.fit(x_train_reduced, y_train)

RandomForestClassifier(random_state=42)

2. Сделайте предикт, посчитайте точность модели и матрицу ошибок. Сравните с предыдущей моделью дерева решений. Есть ли случаи, когда модель из пункта 1 отрабатывает лучше, чем модель случайного леса?

In [191]:
#По анализу матриц ошибок двух вариантов обучения моделей можно сделать вывод, что во всех случаях лучше работает метод "Случайный лес"

In [192]:
# Ваш код здесь 
predicted_train_rf = clf_rf.predict(x_train_reduced)
predicted_test_rf = clf_rf.predict(x_test_reduced)

print(accuracy_score(y_train, predicted_train_rf))
print(accuracy_score(y_test, predicted_test_rf))
print(f'''Дерево решений:
{confusion_matrix(y_test, predicted_test)}''')
print(f'''Случайный лес:
{confusion_matrix(y_test, predicted_test_rf)}''')

1.0
0.7536382536382537
Дерево решений:
[[730  54 213]
 [ 42 696 215]
 [192 229 515]]
Случайный лес:
[[818  38 141]
 [ 17 787 149]
 [160 206 570]]


3. Тюнинг модели случайного леса

Увеличьте точность модели на тестовом датасете RandomForestClassifier c помощью тюнинга параметров. 

Параметры, которые можно настраивать для увеличения точности:
```
    `bootstrap'
    'max_depth'
    'max_features'
    'min_samples_leaf'
    'min_samples_split'
    'random_state'
    'n_estimators'

```

С описанием каждого из параметров можно ознакомиться [в документации](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

Задание засчитывается, если значение метрики строго выше 0.76 на тестовом датасете.

In [194]:
# Ваш код здесь 
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': list(range(100, 501, 100)),
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': list(range(1, 10)),
    'bootstrap': [True, False],
    'random_state': [42, 123, 2023]
}

grid_search_rf = GridSearchCV(
   estimator=clf_rf,
   param_grid=param_grid,
   scoring='accuracy',
   verbose=1,
   n_jobs=-1
)

grid_search_rf.fit(x_train_reduced, y_train)

best_params = grid_search_rf.best_params_
best_params


Fitting 5 folds for each of 540 candidates, totalling 2700 fits


KeyboardInterrupt: 

In [199]:
rf_tuned = RandomForestClassifier(max_features='log2', min_samples_leaf=1, n_estimators=200, bootstrap=False, random_state=42)
rf_tuned.fit(x_train_reduced, y_train)

tuned_test_rf = rf_tuned.predict(x_test_reduced)

print(accuracy_score(y_test, tuned_test_rf))

0.7674982674982676


In [200]:
#param_grid = {
#   'n_estimators': list(range(100, 501, 100)),
#   'max_features': ['sqrt', 'log2'],
#   'min_samples_leaf': list(range(1, 10)),
# эти параметры мой компьютер перебирал минут 20, это 450 операций, если я добавлю еще bootstrap и random_state, то операций станет 2700,
# и мой комп загнется. поэтому эти параметры подставлены методом подбора

4. Анализ влияния фичей на модель

До этого в задачах вы работали над подготовленным датасетом, где фичи были заранее извлечены из текстовых переменных, отскейлены и пропущены через OneHotEncoder. Сравним, какой была бы предсказательная способность модели, если бы мы использовали только сырые данные из исходного датасета. Для этого проделайте следующие шаги:

1. Загрузите датасет `vehicles_dataset_old.csv`.
2. Удалите из него переменную `price` и все строковые колонки. Дерево решений и случайный лес не умеют самостоятельно работать со строковыми значениями.
3. Сформируйте x_train и x_test так же, как они были сформированы в предыдущих заданиях.
4. Обучите свою лучшую модель случайного леса на новых данных и замерьте качество. Убедитесь, что оно ухудшилось.
5. Найдите три фичи, которые лучшим образом повлияли на предсказательную способность модели.

In [202]:
df_old = pd.read_csv('vehicles_dataset_old.csv')
df_old.head()

df_old = df_old.drop(columns=['price'])

df_old = df_old.select_dtypes(exclude=['object'])

print(df_old.columns)

Index(['id', 'year', 'odometer', 'lat', 'long'], dtype='object')


In [203]:
# Ваш код здесь 
x_train1, x_test1, y_train1, y_test1 = train_test_split(df_old, y, test_size=0.3, random_state=42)

rf_tuned.fit(x_train1, y_train1)

predicted_train1 = rf_tuned.predict(x_train1)
predicted_test1 = rf_tuned.predict(x_test1)

print(accuracy_score(y_train1, predicted_train1))
print(accuracy_score(y_test1, predicted_test1))

1.0
0.6309771309771309


In [215]:
f_imp_list1 = list(zip(x_train1.columns, rf_tuned.feature_importances_))
f_imp_list1.sort(key=lambda x: x[1], reverse=True)
f_imp_list1[:3:]

[('odometer', 0.2786532414505148),
 ('year', 0.22088350460875172),
 ('id', 0.16899175819515586)]